In [1]:
from supabase.client import Client
from typing import Dict, Any, Tuple, List
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults


In [2]:
from utils.utils import initialize_environment_variables, initialize_subabase_client, initialize_openai_client
from utils.utils import initialize_tavily_client
from api.config import interface_config

initialize_environment_variables("../.env")

# Initialize Supabase Client
supabase_client = initialize_subabase_client()

# Initialize OpenAI Client
openai_client = initialize_openai_client()

# Initialize Tavily Client
tavily_client = initialize_tavily_client()

In [3]:
input_dict = {
"school_type": "Gymnasium",
"subject": "English",
"topic": "Shakespeare",
"grade": "13.",
"state": "Bayern",
"keywords": ["shakespeare", "poetry", "analysis"], 
"context": ["2147550a-800f-4fd1-9021-e5f443c97ce5", "98e1f41e-5985-418b-a067-aad97091ac23", "26392f41-80ab-4c01-a408-4faf36f06291"]
}

input_dict

In [4]:
def unpack_prompt_input(input: Dict[str, Any]) -> Tuple[str, str, str, str, List[str], List[str]]:
    school_type = str(input.get("subject"))
    subject = str(input.get("subject"))
    topic = str(input.get("topic"))
    grade = str(input.get("grade"))
    state = str(input.get("state"))
    keywords = [str(i) for i in input.get("keywords")]
    context = [str(i) for i in input.get("context")]
    return school_type, subject, topic, grade, state, keywords, context

In [5]:
school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(input_dict)

In [6]:
def create_task_string(school_type, subject, topic, grade, state, keywords, context) -> str:
    keywords = '\n'.join(keywords)
    return f"""Plane eine Unterrichtsstunde im Fach {subject} zum Thema {topic} für eine {grade} Klasse in {state} für den Schultyp {school_type}.
Lege bei dieser Aufgabe einen besonderen Fokus auf die folgenden Schlüsselbegriffe:
    
Schlüsselbegriffe:
------------------
{keywords}
------------------"""

In [7]:
task = create_task_string(school_type, subject, topic, grade, state, keywords, context)
print(task)

Plane eine Unterrichtsstunde im Fach English zum Thema Shakespeare für eine 13. Klasse in Bayern für den Schultyp English.
Lege bei dieser Aufgabe einen besonderen Fokus auf die folgenden Schlüsselbegriffe:
    
Schlüsselbegriffe:
------------------
shakespeare
poetry
analysis
------------------


def get_uploaded_ids(filepath, upload_table_name):
    data, _cnt = supabase_client.table(upload_table_name).select('id').contains('metadata', {'source':filepath}).execute()
    return [data[1][i].get("id") for i in range(len(data[1]))]

def create_filter_list(context):
    uploaded_ids = []
    for doc in context:
        if len(doc) > 0:
            doc_uploaded_ids = get_uploaded_ids(doc, interface_config.upload_table_name)
            if len(doc_uploaded_ids) > 0:
                uploaded_ids += doc_uploaded_ids
    return uploaded_ids

In [8]:
filter_list = context

In [9]:
def initialize_vector_store(client: Client, embedding: OpenAIEmbeddings, table_name: str, query_name: str):
    vector_store = SupabaseVectorStore(
        client=client,
        embedding=embedding,
        table_name=table_name,
        query_name=query_name,
    )
    return vector_store

In [10]:
embedding = OpenAIEmbeddings()

In [11]:
vector_store = initialize_vector_store(supabase_client, embedding, interface_config.upload_table_name, interface_config.supabase_match_function) 

In [12]:
retrieved_context = vector_store.similarity_search(task,k=2,filter=filter_list)

2024-02-03 00:13:36,846:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 00:13:36,991:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=2 "HTTP/1.1 200 OK"


In [13]:
print(type(retrieved_context[0]))

<class 'langchain_core.documents.base.Document'>


In [14]:
retrieved_context[0]

Document(page_content='ENTWURF RICHTLINIEN 02.08.2023    \n 14 Neben diesen Maßnahmen zur individuellen Förderung im kompetenzorientierten Un-\nterricht sind sonderpädagogische Förderpläne ein zentrales Element der Bildungspla-\nnung für Schülerinnen und Schüler mit Bedarf an sonderpädagogischer Unterstützung. \nDer Förderplan als Lern- und Entwicklungsplan konkretisiert den sonderpädagogi-\nschen Unterstützungsbedarf zum Unterricht und macht die daraus abzuleitenden För-\ndermaßnahmen und die Lernfortschritte für alle Beteiligten nachvollziehbar und trans-\nparent. Hierzu kooperieren die Lehrkräfte mit allen anderen an der Förderung beteilig-\nten Personen. Förderpläne im Kontext sonderpädagogischer Förderung sind als In-\nstrument kontinuierlich weiterzuentwickeln und zu aktualisieren. Unabhängig von der \nförmlichen Feststellung eines sonderpädagogischen Unterstützungsbedarfs ist ein in-\ndividueller Förderplan immer dann zu erstellen, wenn die Bedarfe der Schülerin oder \ndes Schül

In [15]:
def create_prompt_template(retrieved_context, task) -> str:
    return f"""Du bist ein Assistent für Lehrende und deine Aufgabe ist Unterricht nach den höchsten Standards vorzubereiten. 
Schlüsselwöter helfen dir bei der Lösung der Aufgabe den Fokus richtig zu wählen.
Die Lehrkraft stellt dir wichtigen Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. 
Die Lösung der Aufgabe soll in jedem Fall eine inhaltliche und zeitliche Struktur - insgesamt 45 min - für den Unterricht beinhalten.
Wenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden.
Die Lösung darf keine Wiederholungen enthalten!

Hier ist ein relevanter Ausschnitt aus dem Kontext aus deiner Bibliothek:

Kontext:
---------
{retrieved_context}
---------

Los geht's!

Die Antwort soll die unten in JSON definierte Tabelle genau ein Mal befüllen. Die Anzahl der table_data Objekte soll an den Inhalt angepasst werden:
______________________________

{{
    "learn_goals": "xyz",
    "table_data": [
        {{"title": "abc", "duration": "10min", "content": "blabla"}},
        {{...}},
        {{...}},
        ...,
    ]
}}
______________________________

Aufgabe: {task}"""
prompt_template = create_prompt_template(retrieved_context, task)

In [16]:
print(prompt_template)

Du bist ein Assistent für Lehrende und deine Aufgabe ist Unterricht nach den höchsten Standards vorzubereiten. 
Schlüsselwöter helfen dir bei der Lösung der Aufgabe den Fokus richtig zu wählen.
Die Lehrkraft stellt dir wichtigen Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. 
Die Lösung der Aufgabe soll in jedem Fall eine inhaltliche und zeitliche Struktur - insgesamt 45 min - für den Unterricht beinhalten.
Wenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden.
Die Lösung darf keine Wiederholungen enthalten!

Hier ist ein relevanter Ausschnitt aus dem Kontext aus deiner Bibliothek:

Kontext:
---------
[Document(page_content='ENTWURF RICHTLINIEN 02.08.2023    \n 14 Neben diesen Maßnahmen zur individuellen Förderung im kompetenzorientierten Un-\nterricht sind sonderpädagogische Förderpläne ein zentrales Element der Bildungspla-\nnung für Schülerinnen und Schüler mit Bedarf an sonderpädagogischer Unte

In [17]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [18]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

In [19]:
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="vector_store",
        query_name="match_documents_with_uuid_filter",
    )
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 4, 'lambda_mult': 0.25, 'filter': filter_list})
retriever_tool = tool = create_retriever_tool(
    retriever,
    "get_more_context",
    "Retrieve more relevant context information.",
)

In [20]:
tools = [retriever_tool, tavily_tool]

In [21]:
llm = ChatOpenAI(model_name=interface_config.gpt_model_version, temperature=interface_config.gpt_model_temperature, response_format={ "type": "json_object" })
agent = create_openai_functions_agent(llm, tools, prompt)

/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:299: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [22]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [23]:
response = agent_executor.invoke({"input": prompt_template})



> Entering new AgentExecutor chain...


2024-02-03 00:13:38,969:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `get_more_context` with `{'query': 'shakespeare poetry analysis'}`




2024-02-03 00:13:39,294:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 00:13:39,357:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=20 "HTTP/1.1 200 OK"


2024-02-03 00:13:39,906:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



{
    "learn_goals": "Die Schülerinnen und Schüler sollen eine Einführung in das Leben und Werk von William Shakespeare erhalten, seine Bedeutung für die englische Literatur verstehen und Gedichte von Shakespeare analysieren.",
    "table_data": [
        {
            "title": "Einführung in das Leben und Werk von Shakespeare",
            "duration": "15min",
            "content": "Kurze Präsentation über das Leben von Shakespeare und Bedeutung seiner Werke für die englische Literatur."
        },
        {
            "title": "Gedichtanalyse",
            "duration": "15min",
            "content": "Gemeinsame Analyse eines Gedichts von Shakespeare, Diskussion über die Themen, Sprache und Stilistik."
        },
        {
            "title": "Kreatives Schreiben",
            "duration": "10min",
            "content": "Kurze Schreibaufgabe, in der die Schülerinnen und Schüler selbst ein Gedicht im Stil von Shakespeare verfassen."
        },
        {
            "title": "Reflex

In [24]:
print(type(retrieved_context[0]))

<class 'langchain_core.documents.base.Document'>


In [25]:
context_dict = [doc.dict() for doc in retrieved_context]

In [26]:
len(context_dict)

2

In [27]:
response["context"] = context_dict

In [28]:
type(response)

dict

In [29]:
response

{'input': 'Du bist ein Assistent für Lehrende und deine Aufgabe ist Unterricht nach den höchsten Standards vorzubereiten. \nSchlüsselwöter helfen dir bei der Lösung der Aufgabe den Fokus richtig zu wählen.\nDie Lehrkraft stellt dir wichtigen Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. \nDie Lösung der Aufgabe soll in jedem Fall eine inhaltliche und zeitliche Struktur - insgesamt 45 min - für den Unterricht beinhalten.\nWenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden.\nDie Lösung darf keine Wiederholungen enthalten!\n\nHier ist ein relevanter Ausschnitt aus dem Kontext aus deiner Bibliothek:\n\nKontext:\n---------\n[Document(page_content=\'ENTWURF RICHTLINIEN 02.08.2023    \\n 14 Neben diesen Maßnahmen zur individuellen Förderung im kompetenzorientierten Un-\\nterricht sind sonderpädagogische Förderpläne ein zentrales Element der Bildungspla-\\nnung für Schülerinnen und Schüler mit Bedarf a

In [30]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser


In [31]:
for (key, value) in response.items():
    print(f"key: {type(key)}")
    print(f"value: {type(value)}")

key: <class 'str'>
value: <class 'str'>
key: <class 'str'>
value: <class 'str'>
key: <class 'str'>
value: <class 'list'>
key: <class 'str'>
value: <class 'list'>


In [32]:
print(type(response))

<class 'dict'>


In [33]:
import json
output = json.dumps(response)

TypeError: Object of type AgentActionMessageLog is not JSON serializable

In [ ]:
output

'{"input": "Du bist ein Assistent f\\u00fcr Lehrende und hilfst ihnen dabei ihren Unterricht vorzubereiten. \\nVerwende dazu die Informationen im Kontext, um die Aufgabe am Ende zu l\\u00f6sen. Gib den Schl\\u00fcsselw\\u00f6tern bei der L\\u00f6sung der Aufgabe ein besonders hohes Gewicht.\\nWenn du keine L\\u00f6sung findest, sage einfach, dass du die Aufgabe nicht l\\u00f6sen kannst, und versuche keine Antwort zu erfinden. \\nDie L\\u00f6sung der Aufgabe soll eine inhaltliche und zeitliche Struktur beinhalten.\\n\\nKontext:\\n---------\\n[Document(page_content=\'ENTWURF RICHTLINIEN 02.08.2023    \\\\n 14 Neben diesen Ma\\u00dfnahmen zur individuellen F\\u00f6rderung im kompetenzorientierten Un-\\\\nterricht sind sonderp\\u00e4dagogische F\\u00f6rderpl\\u00e4ne ein zentrales Element der Bildungspla-\\\\nnung f\\u00fcr Sch\\u00fclerinnen und Sch\\u00fcler mit Bedarf an sonderp\\u00e4dagogischer Unterst\\u00fctzung. \\\\nDer F\\u00f6rderplan als Lern- und Entwicklungsplan konkretisiert

In [ ]:
def download_file_from_bucket(destination: str, full_path: str, path: str):
    full_path_list = full_path.split("/")
    bucket_name = full_path_list[0]
    with open(destination, 'wb+') as f:
        storage_file = supabase_client.storage.from_(bucket_name).download(path)
        f.write(storage_file)

In [ ]:
download_file_from_bucket("RMH-Rechnung_12_23.pdf", "context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf", "user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")

2024-02-01 18:23:05,084:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/storage/v1/object/context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf "HTTP/1.1 200 OK"


In [ ]:
upload_dict = {"path":"user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf","id":"c2f81078-039d-435a-b0c6-84534ac9048f","fullPath":"context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf"}

In [ ]:
full_path = upload_dict['fullPath']

In [ ]:
full_path

'context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf'

In [ ]:
full_path.split("/")

'context_uploads'

In [ ]:
import os
import tempfile
import mimetypes
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename

from api.files_service import allowed_file, download_file_from_bucket, upload_text, upload_pdf, get_uploaded_ids
from api.ai_service import getanswer


file_path = secure_filename("user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")  
print(file_path)  
download_file_from_bucket(file_path, "context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf", "user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")
print(mimetypes.guess_type(file_path))
if mimetypes.guess_type(file_path)[0] == 'text/plain':
    upload_text(file_path)
elif mimetypes.guess_type(file_path)[0] == 'application/pdf':
    print(file_path)
    upload_pdf(file_path)
vector_store_ids = get_uploaded_ids(file_path, interface_config.upload_table_name)


2024-02-02 18:49:17,954:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/storage/v1/object/context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf "HTTP/1.1 200 OK"


user_2e4ace06-270b-429b-9726-f937261ed293_RMH-Rechnung_12_23.pdf
('application/pdf', None)
user_2e4ace06-270b-429b-9726-f937261ed293_RMH-Rechnung_12_23.pdf


TypeError: PyPDFLoader.__init__() got an unexpected keyword argument 'autodetect_encoding'

In [ ]:
{
  "id": "chatcmpl-123",
  "object": "chat.completion",
  "created": 1677652288,
  "model": "gpt-3.5-turbo-0613",
  "system_fingerprint": "fp_44709d6fcb",
  "choices": [{
    "index": 0,
    "message": {
      "role": "assistant",
      "content": "\n\nHello there, how may I assist you today?",
    },
    "logprobs": null,
    "finish_reason": "stop"
  }],
  "usage": {
    "prompt_tokens": 9,
    "completion_tokens": 12,
    "total_tokens": 21
  }
}
